In this file, the goal to create a .csv file containing the chamfer distance for each subject in PEPR_Marseille (including those who were not selected) as the file containing the reconstruction_error

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.ndimage import distance_transform_edt
from numpy.lib.stride_tricks import sliding_window_view

In [2]:
dir_path= "/neurospin/tmp/fred/models/2025-11-10"  

In [3]:
def edge_distance_kernel(mask, distance_threshold=5.0, sigma=2.0):
    """
    Compute a spatial weighting kernel emphasizing the center of a binary mask.
    Inside the mask:
      - Weights = 1 for pixels farther than `distance_threshold` from the edge.
      - Weights = Gaussian decay for pixels within `distance_threshold` of the edge.
    Outside the mask:
      - Weights = 0.
    Parameters
    ----------
    mask : np.ndarray
        2D or 3D binary mask defining the valid region (1 inside, 0 outside).
    distance_threshold : float
        Distance (in pixels/voxels) from the mask edge above which weight = 1.
    sigma : float
        Gaussian falloff width (controls how fast the weight decays near edges).
    Returns
    -------
    kernel : np.ndarray
        Weight map of same shape as `mask`, values in [0,1].
    """
    assert mask.ndim in (2, 3), "mask must be 2D or 3D"
    mask = (mask > 0).astype(np.uint8)
    # Compute distance to the nearest 0 (edge)
    dist_inside = distance_transform_edt(mask)
    # Gaussian decay near the edge
    kernel = np.ones_like(mask, dtype=float)
    near_edge = dist_inside < distance_threshold
    kernel[near_edge] = np.exp(-((distance_threshold - dist_inside[near_edge]) ** 2) / (2 * sigma ** 2))
    # Zero weight outside the mask
    kernel[mask == 0] = 0.0
    return kernel

In [4]:
def chamfer_sweep_weighted(
    binary,
    reconstruction,
    threshold=0.55,
    kernel=None,
    two_sided=True
):
    """
    Compute weighted Chamfer distance between a binary mask and thresholded
    reconstructions over a range of thresholds.
    Parameters
    ----------
    binary : np.ndarray
        2D or 3D binary ground truth (0/1).
    reconstruction : np.ndarray
        2D or 3D continuous reconstruction in [0,1].
    kernel : np.ndarray or None
        Weighting map of same shape as input. If None, use uniform weights = 1.
    two_sided : bool
        Whether to compute symmetric Chamfer (True) or one-sided (False).
    local_window : int or None
        If None, returns mean error per threshold (scalar).
        If int, compute the sum of weighted error in each valid window
        (no padding) and take the maximum as the score.
    """
    assert binary.shape == reconstruction.shape, "Shapes must match"
    assert binary.ndim in (2, 3), "Supports only 2D or 3D arrays"
    binary = (binary > 0.5).astype(np.uint8)
    if kernel is None:
        kernel = np.ones_like(binary, dtype=float)
    else:
        assert kernel.shape == binary.shape, "Kernel must have same shape as input"
    dist_bin = distance_transform_edt(1 - binary)


    recon_bin = (reconstruction > threshold).astype(np.uint8)
    dist_recon = distance_transform_edt(1 - recon_bin)
    if two_sided:
        error_map = binary * dist_recon + recon_bin * dist_bin
    else:
        error_map = recon_bin * dist_bin
    weighted_error = error_map * kernel

    chamfer_score = weighted_error.mean()

    return chamfer_score

In [5]:
def transform(inputs, outputs):

    return inputs[:,0,:,:,:], outputs[:,1,:,:,:]

In [6]:
def transform_mask(mask, input):

    target_shape = input[0].shape

    pad_width = [(0, t - s) for s, t in zip(mask.shape, target_shape)]

    mask = np.pad(mask, pad_width, mode='constant', constant_values=0)
    return mask

In [7]:
mask_path = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/crops/2mm"

In [8]:
mask_path1= "/neurospin/dico/data/deep_folding/current/datasets"

In [9]:
error_PEPR= pd.read_csv("/neurospin/tmp/fred/models/2025-11-10/SC-sylv_right_UKB_16-20-03_123/PEPR_Marseille/Reconstruction_error.csv")

In [11]:
error_PEPR

,Unnamed: 0,0
0,0001BAR_27112016,2398.638672
1,0001BUC_17062014,3461.456299
2,0001FLO_02052012,3145.983887
3,0001FRE_16032015,2839.419434
4,0001GRE_25112014,3589.877930
...,...,...
1030,0269GRE_07122021,4711.532715
1031,0270GRE_22012022,4372.996582
1032,0271GRE_19012022,3350.373291
1033,0272GRE_22032022,3720.735596


In [10]:
import re

In [13]:
subfolders = sorted([
    f for f in os.listdir(dir_path)
    if os.path.isdir(os.path.join(dir_path, f))
])


for i, folder in enumerate(subfolders):
    print(i)
    regions=re.match(r"(.*?(left|right))", folder).group(1)

    current_path = os.path.join(dir_path, folder, "PEPR_Marseille")


    input_PEPR = os.path.join(current_path,"inputs.npy")
    output_PEPR = os.path.join(current_path,"outputs.npy")

    input_PEPR = np.load(input_PEPR)
    output_PEPR = np.load(output_PEPR)

    input_PEPR, output_PEPR= transform(input_PEPR, output_PEPR)

    # DataFrame containing the Chamfer distance to save
    df_Chamfer =pd.DataFrame(columns=["ID", "Chamfer distance"])


    #find the folder in mask_path corresponding to folder
    subfolders_mask = [
    f for f in os.listdir(mask_path)
    if os.path.isdir(os.path.join(mask_path, f))
]
    
    for folder_mask in subfolders_mask:
        folder_clean = folder_mask.replace(".", "")  
        if folder.startswith(folder_clean):
            match = folder_mask
            break
    
    if "right" in folder:
        side_mask="Rmask.npy"
    elif "left" in folder:
        side_mask="Lmask.npy"

    mask=os.path.join(mask_path, match, "mask", side_mask)
    mask= np.load(mask)
    mask= mask[:,:,:,0]
    mask= transform_mask(mask,input_PEPR)
    kernel=edge_distance_kernel(mask)


    for j in range(input_PEPR.shape[0]):
        chamfer_PEPR= chamfer_sweep_weighted(input_PEPR[j,:,:,:], output_PEPR[j,:,:,:],0.55, kernel)
        df_Chamfer.loc[j] = [error_PEPR.iloc[j,0], chamfer_PEPR]


    df_Chamfer.to_csv(os.path.join(current_path, "Chamfer_distance.csv"))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [1]:
import pandas as pd 

In [2]:
df= pd.read_csv("/neurospin/tmp/fred/models/2025-11-10/FCLp-subsc-FCLa-INSULA_left_UKB_15-59-28_107/PEPR_Marseille/Chamfer_distance.csv")

In [3]:
df

,Unnamed: 0,ID,Chamfer distance
0,0,0001BAR_27112016,0.006064
1,1,0001BUC_17062014,0.021547
2,2,0001FLO_02052012,0.009219
3,3,0001FRE_16032015,0.102376
4,4,0001GRE_25112014,0.170339
...,...,...,...
1030,1030,0269GRE_07122021,0.019660
1031,1031,0270GRE_22012022,0.009172
1032,1032,0271GRE_19012022,0.011784
1033,1033,0272GRE_22032022,0.007307


In [5]:
import numpy as np

In [6]:
selected_index = np.load("/neurospin/dico/fred/Runs/01_betaVAE/Program/2023_jlaval_STSbabies/betaVAE/notebooks/fred/PEPR_Marseille/All the subjects/histogram_non_zero/index_to_save.npy")

In [7]:
df = df.loc[selected_index].reset_index(drop=True)
df

,Unnamed: 0,ID,Chamfer distance
0,0,0001BAR_27112016,0.006064
1,2,0001FLO_02052012,0.009219
2,6,0001HUH_30112018,0.012770
3,7,0001KCH_08042010,0.007436
4,8,0001LYO_19012015,0.046545
...,...,...,...
820,1027,0265GRE_08102013,0.012225
821,1028,0266GRE_29092021,0.008910
822,1029,0268GRE_30112021,0.009642
823,1033,0272GRE_22032022,0.007307


In [8]:
df = df.iloc[:,1]
df

0      0001BAR_27112016
1      0001FLO_02052012
2      0001HUH_30112018
3      0001KCH_08042010
4      0001LYO_19012015
             ...       
820    0265GRE_08102013
821    0266GRE_29092021
822    0268GRE_30112021
823    0272GRE_22032022
824    0273GRE_30032022
Name: ID, Length: 825, dtype: object